In [1]:
# Import libraries
import pandas as pd
import os
from pathlib import Path
from tqdm import tqdm
import yaml
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

In [2]:
LEARNED_THR = 0.
IOU_THR = 0.6
PREDICTION_PATH = "dataset/predictions/03-predictions.csv"

In [3]:
PREDICTION_IMAGE_PATH = Path("dataset/images/test")

test_df = pd.read_csv(PREDICTION_PATH)
test_df.sample(5)

,Image_ID,class,confidence,ymin,xmin,ymax,xmax
155178,ID_F7a5DB.jpg,cssvd,0.000561,423.185577,945.705627,810.000000,1079.637573
472824,ID_lDkNRk.jpeg,anthracnose,0.003455,1092.708862,0.000000,2500.822021,398.671082
319830,ID_NoKFT1.jpeg,healthy,0.007889,20.386606,0.000000,2343.698486,395.858826
13384,ID_iKbbSX.jpg,cssvd,0.000200,6.337910,150.339249,562.031555,476.028015
137223,ID_QgwdlL.jpg,cssvd,0.000347,701.315430,84.036774,1292.089722,453.381134


In [4]:
class_id = {j: i for i, j in enumerate(sorted(test_df["class"].unique()))}
test_df["class_id"] = test_df["class"].map(class_id)
class_id

{'anthracnose': 0, 'cssvd': 1, 'healthy': 2}

In [5]:
test_df.head()

,Image_ID,class,confidence,ymin,xmin,ymax,xmax,class_id
0,ID_cWEAQI.jpeg,healthy,0.544732,0.000000,0.000000,3671.547607,1677.223511,2
1,ID_cWEAQI.jpeg,healthy,0.011575,319.800598,1257.447021,1133.294067,1794.507324,2
2,ID_cWEAQI.jpeg,cssvd,0.010274,2121.080322,25.210379,3996.958008,1582.924072,1
3,ID_cWEAQI.jpeg,healthy,0.008656,1568.007812,16.477583,3980.346924,1614.395996,2
4,ID_cWEAQI.jpeg,anthracnose,0.007832,2376.124756,12.456416,3995.283447,1585.965576,0


In [6]:
import torch

def convert_df(df: pd.DataFrame):
	df = df.copy().dropna()
	return {
		img_id: {
			"boxes": torch.tensor(raw[["xmin", "ymin", "xmax", "ymax"]].values, dtype=torch.float32),
			"scores": (
				torch.tensor(raw["confidence"].values, dtype=torch.float32)
				if "confidence" in raw.columns
				else None
			),
			"labels": torch.tensor(raw["class_id"].values, dtype=torch.int32),
		}
		for (img_id, ), raw in df.groupby(["Image_ID"])
	}

def default_value():
	return {
		"boxes": torch.empty((0, 4), dtype=torch.float32),
		"scores": torch.empty((0,), dtype=torch.float32),
		"labels": torch.empty((0,), dtype=torch.int32),
	}


converted_labels = list(PREDICTION_IMAGE_PATH.glob("*"))
converted_labels = [i.name for i in converted_labels]

def get_preds_data(preds, thr: float = 0.5):
	if thr is not None:
		preds = preds[preds["confidence"] >= thr]
	preds = convert_df(preds)
	print(len(preds))
	print(list(preds.keys())[:10])
	d = default_value()
	return {i: preds.get(i, d) for i in converted_labels}

In [7]:
test_df.sample(2)

,Image_ID,class,confidence,ymin,xmin,ymax,xmax,class_id
248780,ID_DldPpd.jpeg,anthracnose,0.000589,0.000000,0.0,390.602112,1583.528320,0
259175,ID_R6trN8.jpeg,healthy,0.000184,868.674805,0.0,1178.749268,73.801819,2


In [8]:
predictions = get_preds_data(test_df, thr=None)

len(predictions)

1626
['ID_A16nzu.jpg', 'ID_A1Euyz.jpg', 'ID_A1HcV0.jpeg', 'ID_A4ZdJC.jpeg', 'ID_A5SFUW.jpeg', 'ID_A6Fogi.jpeg', 'ID_ABDCyn.jpeg', 'ID_ACg6Qf.jpeg', 'ID_AFi8A1.jpg', 'ID_AHlc9P.jpg']


1626

In [9]:
predictions["ID_CGnAYP.jpg"]

{'boxes': tensor([[0.0000e+00, 2.1876e+01, 7.4241e+02, 1.2184e+03],
         [5.3887e+02, 2.0360e+00, 9.6000e+02, 1.1668e+03],
         [7.0636e+02, 2.9759e+01, 9.6000e+02, 1.1975e+03],
         ...,
         [5.9015e-01, 1.0975e+03, 1.0411e+02, 1.2789e+03],
         [4.4907e+00, 1.1791e+03, 4.1567e+02, 1.2798e+03],
         [4.1821e+02, 5.6394e+02, 6.6524e+02, 9.5435e+02]]),
 'scores': tensor([6.4830e-01, 6.1857e-01, 4.8890e-02, 2.3590e-02, 1.6681e-02, 1.6367e-02,
         1.4344e-02, 1.0209e-02, 8.9888e-03, 8.9760e-03, 7.5501e-03, 6.9781e-03,
         6.8132e-03, 5.3830e-03, 5.3579e-03, 5.2456e-03, 4.7497e-03, 4.7260e-03,
         4.2811e-03, 4.1942e-03, 3.7699e-03, 3.6534e-03, 3.3593e-03, 2.8375e-03,
         2.8077e-03, 2.8014e-03, 2.7688e-03, 2.5962e-03, 2.3090e-03, 2.2306e-03,
         2.1974e-03, 2.1328e-03, 1.9978e-03, 1.9747e-03, 1.9265e-03, 1.9119e-03,
         1.9051e-03, 1.8457e-03, 1.8430e-03, 1.8230e-03, 1.7043e-03, 1.6489e-03,
         1.6131e-03, 1.5878e-03, 1.2481e-03,

In [10]:
predictions["ID_CGnAYP.jpg"].keys()

dict_keys(['boxes', 'scores', 'labels'])

In [11]:
from torchvision.ops import nms

def apply_nms(predictions: dict[str, torch.Tensor], iou_threshold=0.5):
    """
    Apply Non-Maximum Suppression (NMS) to reduce overlapping boxes.

    Args:
        predictions (dict): Dictionary containing keys ['boxes', 'scores', 'labels'].
        iou_threshold (float): Intersection over Union (IoU) threshold for NMS.

    Returns:
        dict: Filtered predictions after applying NMS.
    """
    filtered_predictions = {}
    for image_id, data in predictions.items():
        boxes = data["boxes"]
        scores = data["scores"]
        labels = data["labels"]

        if boxes.numel() == 0:
            # If no boxes, skip this image
            filtered_predictions[image_id] = data
            continue

        # Perform NMS for each class separately
        keep_indices = []
        for label in labels.unique():
            label_mask = labels == label
            label_boxes = boxes[label_mask]
            label_scores = scores[label_mask]
            indices = nms(label_boxes, label_scores, iou_threshold)
            keep_indices.extend(label_mask.nonzero(as_tuple=True)[0][indices].tolist())

        # Filter boxes, scores, and labels based on NMS results
        keep_indices = torch.tensor(keep_indices, dtype=torch.long)
        filtered_predictions[image_id] = {
            "boxes": boxes[keep_indices],
            "scores": scores[keep_indices],
            "labels": labels[keep_indices],
        }

    return filtered_predictions

# Apply NMS to predictions
filtered_predictions = apply_nms(predictions, iou_threshold=IOU_THR)

In [12]:
for k, v in predictions.items():
    r = filtered_predictions[k]
    if len(v["boxes"]) != len(r["boxes"]):
        print(k, len(v["boxes"]), len(r["boxes"]))
        print("Boxes before NMS:", v["boxes"])
        print("Boxes after NMS:", r["boxes"])
        print("Scores before NMS:", v["scores"])
        print("Scores after NMS:", r["scores"])
        print("Labels before NMS:", v["labels"])
        print("Labels after NMS:", r["labels"])
        print()
        break

ID_cWEAQI.jpeg 300 148
Boxes before NMS: tensor([[0.0000e+00, 0.0000e+00, 1.6772e+03, 3.6715e+03],
        [1.2574e+03, 3.1980e+02, 1.7945e+03, 1.1333e+03],
        [2.5210e+01, 2.1211e+03, 1.5829e+03, 3.9970e+03],
        ...,
        [9.8768e+02, 3.2461e+03, 1.2334e+03, 3.6635e+03],
        [1.8305e+00, 2.5417e+03, 3.1909e+02, 3.7669e+03],
        [1.6164e+02, 0.0000e+00, 6.1932e+02, 9.4719e+01]])
Boxes after NMS: tensor([[1.2456e+01, 2.3761e+03, 1.5860e+03, 3.9953e+03],
        [4.5787e+00, 3.1392e+03, 9.6250e+02, 4.0000e+03],
        [1.0722e+01, 2.4717e+03, 1.0160e+03, 4.0000e+03],
        [1.2528e+03, 3.4582e+02, 1.8000e+03, 1.1889e+03],
        [0.0000e+00, 3.4323e+03, 1.5875e+03, 3.9998e+03],
        [6.0938e+02, 2.7390e+03, 1.3639e+03, 3.9408e+03],
        [6.7687e+02, 2.4929e+03, 1.5484e+03, 4.0000e+03],
        [4.6578e+02, 2.2093e+00, 1.7928e+03, 2.1906e+02],
        [5.5630e+02, 1.0407e+00, 1.8000e+03, 1.1274e+03],
        [6.1108e+02, 4.6271e-01, 1.7273e+03, 3.2681e+02],


In [13]:
filtered_data = []
id_to_label = {v: k for k, v in class_id.items()}
for image_id, data in filtered_predictions.items():
    boxes = data["boxes"].tolist()
    scores = data["scores"].tolist()
    labels = data["labels"].tolist()
    for box, score, label in zip(boxes, scores, labels):
        filtered_data.append({
            "Image_ID": image_id,
            "class": id_to_label[label],
            "confidence": score,
            "ymin": box[1],
            "xmin": box[0],
            "ymax": box[3],
            "xmax": box[2],
            "class_id": label
        })

filtered_df = pd.DataFrame(filtered_data)
filtered_df.head()

,Image_ID,class,confidence,ymin,xmin,ymax,xmax,class_id
0,ID_cWEAQI.jpeg,anthracnose,0.007832,2376.124756,12.456416,3995.283447,1585.965576,0
1,ID_cWEAQI.jpeg,anthracnose,0.002389,3139.214355,4.578709,4000.000000,962.504578,0
2,ID_cWEAQI.jpeg,anthracnose,0.002003,2471.707275,10.721802,4000.000000,1015.964233,0
3,ID_cWEAQI.jpeg,anthracnose,0.001522,345.821594,1252.781128,1188.937134,1800.000000,0
4,ID_cWEAQI.jpeg,anthracnose,0.001200,3432.309814,0.000000,3999.847168,1587.487427,0


In [14]:
len(filtered_df), len(test_df)

(245351, 487800)

In [15]:
filtered_df["class"].value_counts()

class
cssvd          85880
healthy        84665
anthracnose    74806
Name: count, dtype: int64

In [17]:
files = list(PREDICTION_IMAGE_PATH.glob("*"))

In [19]:
test_df: pd.DataFrame = filtered_df.copy()

In [20]:
test_records = test_df.to_dict(orient="records")

test_records[:5]

[{'Image_ID': 'ID_cWEAQI.jpeg',
  'class': 'anthracnose',
  'confidence': 0.007831727154552937,
  'ymin': 2376.124755859375,
  'xmin': 12.456416130065918,
  'ymax': 3995.283447265625,
  'xmax': 1585.965576171875,
  'class_id': 0},
 {'Image_ID': 'ID_cWEAQI.jpeg',
  'class': 'anthracnose',
  'confidence': 0.0023892666213214397,
  'ymin': 3139.21435546875,
  'xmin': 4.578709125518799,
  'ymax': 4000.0,
  'xmax': 962.5045776367188,
  'class_id': 0},
 {'Image_ID': 'ID_cWEAQI.jpeg',
  'class': 'anthracnose',
  'confidence': 0.00200273166410625,
  'ymin': 2471.707275390625,
  'xmin': 10.7218017578125,
  'ymax': 4000.0,
  'xmax': 1015.9642333984375,
  'class_id': 0},
 {'Image_ID': 'ID_cWEAQI.jpeg',
  'class': 'anthracnose',
  'confidence': 0.0015218722401186824,
  'ymin': 345.82159423828125,
  'xmin': 1252.7811279296875,
  'ymax': 1188.9371337890625,
  'xmax': 1800.0,
  'class_id': 0},
 {'Image_ID': 'ID_cWEAQI.jpeg',
  'class': 'anthracnose',
  'confidence': 0.001199536956846714,
  'ymin': 343

In [21]:
file_id = [i.name for i in files]
file_id[:2]

['ID_cWEAQI.jpeg', 'ID_NtqErb.jpg']

In [22]:
file_okay = set(test_df["Image_ID"].values)
file_nokay = set(file_id) - file_okay

for f in file_nokay:
    test_records.append(
		{
			"Image_ID": f,
		}
	)

In [23]:
test_df = pd.DataFrame(test_records)

test_df.sample(10)

,Image_ID,class,confidence,ymin,xmin,ymax,xmax,class_id
164217,ID_erqIoE.jpg,anthracnose,0.034418,209.156067,0.741516,768.669922,467.945129,0
206637,ID_yEGJej.jpg,cssvd,0.004852,1601.685181,634.553711,2499.729980,1264.223511,1
178059,ID_sWApFk.jpeg,anthracnose,0.001324,1857.760132,3705.796143,2686.987305,4031.522461,0
200686,ID_BiL2B8.jpeg,healthy,0.000203,45.601883,303.890930,774.002563,563.459229,2
91784,ID_msd0pU.jpg,anthracnose,0.000460,68.107834,229.881973,284.737274,439.994507,0
54750,ID_Yc3rtZ.jpg,cssvd,0.000510,0.181466,886.576965,82.282822,1318.219727,1
148964,ID_YqssVm.jpg,healthy,0.001162,612.143616,851.917114,867.105469,960.000000,2
80572,ID_XPNfsS.jpg,cssvd,0.000121,0.000000,538.516235,32.797031,781.661072,1
9148,ID_Frr32r.jpg,cssvd,0.000148,6.534348,111.273155,612.292358,412.427490,1
228542,ID_rPGfuC.jpg,cssvd,0.002533,0.000000,639.537048,351.010925,810.000000,1


In [24]:
test_df["class"].value_counts()

class
cssvd          85880
healthy        84665
anthracnose    74806
Name: count, dtype: int64

In [25]:
test_df.to_csv(PREDICTION_PATH.replace("-prediction", "-submission"), index=False)

In [26]:
PREDICTION_PATH.replace("-prediction", "-submission")

'dataset/predictions/03-submissions.csv'